In [2]:
import dash
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

KeyboardInterrupt: 

In [1]:
CONF_URL = 'D://Book1.csv'


In [3]:
covid_conf_ts = pd.read_csv(CONF_URL)
covid_tested_ts = pd.read_csv(CONF_URL)
covid_recv_ts = pd.read_csv(CONF_URL)
covid_des_ts = pd.read_csv(CONF_URL)

NameError: name 'pd' is not defined

Data Processing

In [223]:
#get data in cleaned time series format for country
def process_data(data,cntry='Germany',window=3):
    conf_ts = data
    conf_ts_cntry = conf_ts[conf_ts['country']==cntry]
    final_dataset = conf_ts_cntry.T[1:5].sum(axis='columns').diff().rolling(window=window).mean()[:]
    df = pd.DataFrame(final_dataset,columns=['Total'])
    return df

In [224]:
#get overall wordlwide total for confirmed, recovered and dead cases
def get_overall_total(df):
    return df.iloc[:,1].sum()

def get_overall_tested(df):
    return df.iloc[:,2].sum()

def get_overall_rec(df):
    return df.iloc[:,3].sum()

def get_overall_des(df):
    return df.iloc[:,4].sum()

conf_overall_total = get_overall_total(covid_conf_ts)
conf_overall_tested = get_overall_tested(covid_tested_ts)
conf_overall_rec = get_overall_rec(covid_conf_ts)
conf_overall_des = get_overall_des(covid_conf_ts)

print('Overall infected:',conf_overall_total)
print('Overall tested:',conf_overall_tested)
print('Overall rec:',conf_overall_rec)
print('Overall des:',conf_overall_des)


Overall infected: 235156822.0
Overall tested: 528011977.0
Overall rec: 73488972.0
Overall des: 3639224.0


In [225]:
#get total for confirmed, recovered and dead for country 
def get_cntry_total(df,cntry='Germany'):
    return df[df['country']==cntry].iloc[:,1].sum()

def get_cntry_tested(df,cntry='Germany'):
    return df[df['country']==cntry].iloc[:,2].sum()

def get_cntry_rec(df,cntry='Germany'):
    return df[df['country']==cntry].iloc[:,3].sum()

def get_cntry_des(df,cntry='Germany'):
    return df[df['country']==cntry].iloc[:,4].sum()

cntry = 'Germany'
conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
conf_cntry_tested = get_cntry_tested(covid_tested_ts,cntry)
conf_cntry_rec = get_cntry_rec(covid_recv_ts,cntry)  
conf_cntry_des = get_cntry_des(covid_des_ts,cntry)

print(f'{cntry} Confirmed:',conf_cntry_total)
print(f'{cntry} Tested:',conf_cntry_tested)
print(f'{cntry} rec:',conf_cntry_rec)
print(f'{cntry} des:',conf_cntry_des)

Germany Confirmed: 9035795.0
Germany Tested: 0.0
Germany rec: 0.0
Germany des: 117126.0


In [294]:
def fig_world_trend(cntry='Algeria',window=1):
    df = process_data(data=covid_conf_ts,cntry=cntry,window=window)
    df.head(10)
    if window==1:
        yaxis_title = "Daily Cases"
    else:
        yaxis_title = "Daily Cases ({}-day MA)".format(window)
    fig = px.area(df, y='Total', x=df.index, title='Daily confirmed cases trend for {}'.format(cntry),height=600,color_discrete_sequence =['maroon'])
    fig.update_layout(title_x=0,plot_bgcolor='#F2DFCE',paper_bgcolor='#F2DFCE',xaxis_title="Data",yaxis_title=yaxis_title)
    return fig

In [277]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

In [278]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Covid-19 Dashboard'

In [279]:
colors = {
    'background': '#111111',
    'bodyColor':'#F2DFCO',
    'text': '#7FDBFF'
}
def get_page_heading_style():
    return {'backgroundColor': colors['background']}


def get_page_heading_title():
    return html.H1(children='COVID-19 Dashboard',
                                        style={
                                        'textAlign': 'center',
                                        'color': colors['text']
                                    })

def get_page_heading_subtitle():
    return html.Div(children='Visualize Covid-19 data generated from sources all over the world.',
                                         style={
                                             'textAlign':'center',
                                             'color':colors['text']
                                         })

def generate_page_header():
    main_header =  dbc.Row(
                            [
                                dbc.Col(get_page_heading_title(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    subtitle_header = dbc.Row(
                            [
                                dbc.Col(get_page_heading_subtitle(),md=15)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    header = (main_header,subtitle_header)
    return header

In [280]:
def get_country_list():
    return covid_conf_ts['country'].unique()

def create_dropdown_list(cntry_list):
    dropdown_list = []
    for cntry in sorted(cntry_list):
        tmp_dict = {'label':cntry,'value':cntry}
        dropdown_list.append(tmp_dict)
    return dropdown_list

def get_country_dropdown(id):
    return html.Div([
                        html.Label('Select Country'),
                        dcc.Dropdown(id='my-id'+str(id),
                            options=create_dropdown_list(get_country_list()),
                            value='Germany'
                        ),
                        html.Div(id='my-div'+str(id))
                    ])

In [295]:
def graph1():
    return dcc.Graph(id='graph1',figure=fig_world_trend('Germany'))

In [290]:
def generate_card_content(card_header,card_value,overall_value):
    card_head_style = {'textAlign':'center','fontSize':'150%'}
    card_body_style = {'textAlign':'center','fontSize':'200%'}
    card_header = dbc.CardHeader(card_header,style=card_head_style)
    card_body = dbc.CardBody(
        [
            html.H5(f"{int(card_value):,}", className="card-title",style=card_body_style),
            html.P(
                "Worlwide: {:,}".format(overall_value),
                className="card-text",style={'textAlign':'center'}
            ),
        ]
    )
    card = [card_header,card_body]
    return card

In [291]:
def generate_cards(cntry='Germany'):
    conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
    cards = html.Div(
        [
            dbc.Row(
                [
                   
                    
                    dbc.Col(dbc.Card(generate_card_content("Tested",conf_cntry_tested,conf_overall_tested), color="success", inverse=True),md=dict(size=2,offset=3)),
                    dbc.Col(dbc.Card(generate_card_content("Recovered",conf_cntry_rec,conf_overall_rec), color="warning", inverse=True),md=dict(size=2)),
                    dbc.Col(dbc.Card(generate_card_content("Infected",conf_cntry_total,conf_overall_total), color="warning", inverse=True),md=dict(size=2)),
                    dbc.Col(dbc.Card(generate_card_content("des",conf_cntry_des,conf_overall_des),color="dark", inverse=True),md=dict(size=2)),
                   
                ],
                className="mb-5",
            ),
        ],id='card1'
    )
    return cards

In [296]:
def generate_layout():
    page_header = generate_page_header()
    layout = dbc.Container(
        [
            page_header[0],
            page_header[1],
            html.Hr(),
            generate_cards(),
            html.Hr(),
            dbc.Row(
                [
                    dbc.Col(get_country_dropdown(id=1),md=dict(size=4,offset=4))                    
                ]
            
            ),
            dbc.Row(
                [                
                    
                    dbc.Col(graph1(),md=dict(size=6,offset=3))
        
                ],
                align="center",

            ),
        ],fluid=True,style={'backgroundColor': colors['bodyColor']}
    )
    return layout

In [297]:
app.layout = generate_layout()


In [298]:
@app.callback(
    [Output(component_id='graph1',component_property='figure'), #line chart
    Output(component_id='card1',component_property='children')], #overall card numbers
    [Input(component_id='my-id1',component_property='value'), #dropdown
     Input(component_id='my-slider',component_property='value')] #slider
)
def update_output_div(input_value1,input_value2):
    return fig_world_trend(input_value1,input_value2),generate_cards(input_value1)

In [299]:
app.run_server(host= '127.0.0.1',debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
